In [1]:
# =====================================
# IMPORTS Y LIBRERIAS:
# =====================================

import pandas as pd
import numpy as np
from statsbombpy import sb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from numpy.linalg import norm
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from collections import Counter
from sklearn.model_selection import KFold  
from IPython.display import display        

In [2]:
# ========================================
# LECTURA DE DATOS:
# ========================================
ENGINE = "pyarrow" # motor a utilizar al leer archivos Parquet
SEASONS = [(317,"2024/2025"), (281,"2023/2024"), (235,"2022/2023"), (108,"2021/2022")] # lista de tuplas (sid, sname)

# Jugadores: estadisticas de jugadores por temporada
statsJugadorPorTemporada = {
    (sid, sname): pd.read_parquet(f"../data/Jugadores/player_season_stats_{sid}.parquet", engine=ENGINE)
    for (sid, sname) in SEASONS
}

# Partidos: partidos por temporada
partidos = {
    (sid, sname): pd.read_parquet(f"../data/Partidos/matches_competition_73_season_{sid}.parquet", engine=ENGINE)
    for (sid, sname) in SEASONS
}

# Equipos: lista de equipos por temporadas
equipos = {
    (sid, sname): partidos[(sid, sname)]["home_team"].dropna().unique().tolist()
    for (sid, sname) in SEASONS
}

FileNotFoundError: [Errno 2] No such file or directory: '../data/Jugadores/player_season_stats_317.parquet'

In [ ]:
# ========================================
# FILTRADO DE DATOS:
# ========================================
# lista con metricas numéricas
METRICS = [
    "player_season_passing_ratio",                      # relacion de número de pases de un jugador respecto al total de pases de su equipo
    "player_season_pass_length",                        # longitud promedio de los pases de un jugador en la temporada
    "player_season_lbp_completed_90",                   # pases que rompen líneas por cada 90 minutos jugados
    "player_season_op_passes_into_box_90",              # pases ofensivos del jugador entraron al área del adversario
    "player_season_np_shots_90",                        # tiros de juego abierto que no son penaltis
    "player_season_np_xg_90",                           # non-penalty expected goals
    "player_season_key_passes_90",                      # pases que crean una oportunidad de tiro para un compañero
    "player_season_xa_90",                              # valor de la probabilidad de que un pase termine en gol
    "player_season_touches_inside_box_90",              # número de toques que tiene un jugador dentro del área del oponente
    "player_season_pressures_90",                       # número de veces que el jugador aplica presion a un adversario
    "player_season_tackles_90",                         # número de tackles que realiza el jugador
    "player_season_interceptions_90",                   # número de intercepciones que realiza el jugador
    "player_season_aerial_wins_90",                     # número de duelos áereos que el jugador gana
    "player_season_padj_tackles_and_interceptions_90",  # número de intercepciones y tackles combinado ajustada por posesión
    "player_season_dribbles_90",                        # número de regates que realiza el jugador
    "player_season_carries_90",                         # número de desplazamientos con balon que realiza el jugador
    "player_season_deep_progressions_90",               # acciones que progresan el balón hacia zonas profundas del oponente
]
ID_COLS = ["player_id","player_name","team_id","team_name","competition_id","season_id","season_name"] # columnas identificadoras que deben acompañar a las metricas
MINUTES_CANDIDATES = ["player_season_minutes","minutes","player_minutes","player_season_360_minutes"] # posibles nombres que representar minutos jugados

def _first_existing(colnames, candidates):
    # helper que dado un iterable de nombres de columnas y una lista de candidatos, devuelve el primer candidato que exista en las columnas
    for c in candidates:
        if c in colnames: return c
    return None

def concat_player_season_stats(statsJugadorPorTemporada, competition_id=73):
    # helper que concatena los dataFrames de estadísticas por temporadas y homologa las columnas
    frames = [] # lista que acumula los DataFrames
    for (season_id, season_name), df in statsJugadorPorTemporada.items():
        # itera sobre el diccionario
        tmp = df.copy()
        if "competition_id" not in tmp.columns: tmp["competition_id"] = competition_id # si falta id, la crea y la rellena con el valor por default
        if "season_id" not in tmp.columns: tmp["season_id"] = season_id # si falta id, la crea y la llena con la clave de la iteración
        if "season_name" not in tmp.columns: tmp["season_name"] = season_name # si falta el nombre, la crea y la rellena con el nombre de la temporada
        rename_map = {}
        if "name" in tmp.columns and "player_name" not in tmp.columns: rename_map["name"] = "player_name" # si la columna se llama name y no existe player_name prepara renombrado
        if "team" in tmp.columns and "team_name" not in tmp.columns: rename_map["team"] = "team_name" # si la columna se llama team y no existe team_name prepara renombrado
        if "teamId" in tmp.columns and "team_id" not in tmp.columns: rename_map["teamId"] = "team_id" # si la columna se llama teamId y no existe team_id prepara renombrado
        if "playerId" in tmp.columns and "player_id" not in tmp.columns: rename_map["playerId"] = "player_id" # si la columna se llama playerId y no existe player_id prepara renombrado
        if rename_map: tmp = tmp.rename(columns=rename_map) # si renombra utiliza DataFrame.rename
        frames.append(tmp) # agrega el dataframe estandarizado de esas temporada a la lista frames
    if not frames: return pd.DataFrame() # si no se agrego nada, devuelve un DataFrame vacío
    all_stats = pd.concat(frames, axis=0, ignore_index=True) # garantiza que exista la métrica si no, la crea con 0.0
    if "player_season_lbp_completed_90" not in all_stats.columns:
        # garantiza que exista la métrica player_season_lbp_completed_90
        all_stats["player_season_lbp_completed_90"] = 0.0
    for c in ["team_id","team_name"]:
        # garantiza la presencia de team_id y team_name si falta regresa NaN
        if c not in all_stats.columns: all_stats[c] = np.nan
    for m in METRICS:
        # recorre cada métrica
        if m in all_stats.columns: all_stats[m] = pd.to_numeric(all_stats[m], errors="coerce") # si existe convierte su tipo a numérico 
        else: all_stats[m] = np.nan # si no existe crea la columna con NaN
    return all_stats # devuelve el DataFrame concatenado y normalizado

def filter_by_minutes(df, min_minutes=900):
    # función para filtrar jugadores por un umbral mínimo de minutos jugados
    min_col = _first_existing(df.columns, MINUTES_CANDIDATES) # busca cual de los nombres candidatos existe en el df
    if min_col is None: return df.copy() # si no hay ninguna columna de minutos, devuelve copias sin filtrar
    return df[df[min_col] >= min_minutes].copy() # filtra filas donde los minutos en min_col >= min_minutes y devuelve la copia

'\ndef build_fingerprint_df(statsJugadorPorTemporada, competition_id=73, min_minutes=900,\n                         n_components=3, group_standardization=("season_id",)):\n    df = concat_player_season_stats(statsJugadorPorTemporada, competition_id=competition_id)\n    df = filter_by_minutes(df, min_minutes=min_minutes)\n    df["player_season_lbp_completed_90"] = df["player_season_lbp_completed_90"].fillna(0)\n    Z_parts, scaler_by_group = [], {}\n    gcols = list(group_standardization) if group_standardization else []\n    if gcols:\n        for gkey, gdf in df.groupby(gcols, dropna=False):\n            X = gdf[METRICS].copy()\n            med = X.median(numeric_only=True)\n            X = X.fillna(med)\n            scaler = StandardScaler()\n            Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=gdf.index)\n            gZ = pd.concat([gdf[ID_COLS], Xz.add_prefix("z_")], axis=1)\n            Z_parts.append(gZ)\n            scaler_by_group[gkey if isinstance(gke

In [ ]:
# ===============================
# FILTRADO DE POSICIONES:
# ===============================
GK_SYNONYMS = {"goalkeeper", "gk", "portero", "arquero", "guardameta", "goal keeper"} # conjunto de strings que identifica los diferentes nombres para la posicion de portero

def _norm_pos(s: str) -> str:
    # Normaliza etiquetas de posición (para cubrir "Goalkeeper", "GK", etc.)
    s = str(s).strip().lower() # asegura que s sea string, quita espacios y pone todo en minúsculas
    s = s.replace("-", " ").replace("_", " ") # unifica los separados
    return s # regresa la string normalizada

def attach_positions(df_stats: pd.DataFrame, pos_csv_path: str) -> pd.DataFrame:
    """
    Une (season_id, player_id) con posiciones desde tu CSV externo.
    Espera columnas: season_id, player_id, position
    Crea columna 'position_norm' normalizada a minúsculas.
    """
    pos = pd.read_csv(pos_csv_path, usecols=["season_id", "player_id", "position"]) # lectura de las columnas necesarias del CSV con posiciones
    pos["position_norm"] = pos["position"].map(_norm_pos) # crea una columna normalizada aplicando _norm_pos a position
    out = df_stats.merge(
        # left join entre df_stats y el DF de posiciones
        pos[["season_id", "player_id", "position_norm"]],
        on=["season_id", "player_id"],
        how="left"
    )
    return out # regresa el DF eriquecido con la normalización de posiciones

def drop_goalkeepers(df_stats_with_pos: pd.DataFrame) -> pd.DataFrame:
    """
    Elimina SOLO si sabemos que es portero (position_norm ∈ GK_SYNONYMS).
    Si viene NaN (desconocido), LO DEJAMOS.
    """
    return df_stats_with_pos.loc[~df_stats_with_pos["position_norm"].isin(GK_SYNONYMS)].copy() # filtra manteniendo únicamente filas cuya position_norm no sea el portero.

In [ ]:
# ===== Builder: igual que tu build_fingerprint_df_autoK, pero parte de un DF ya filtrado =====
def build_fingerprint_df_autoK_FROM_DF(df_input: pd.DataFrame,
                                       k_method="parallel",
                                       variance_threshold=0.85,
                                       group_standardization=("season_id",),
                                       random_state=42):
    """
    Construye Zall (z-scores + PCA auto-K) tomando un DataFrame YA FILTRADO (sin porteros).
    Respeta tus METRICS/ID_COLS y la normalización por grupo (season_id).
    """
    # 1) Copia defensiva y asegura columnas
    df = df_input.copy()
    for c in ID_COLS:
        if c not in df.columns:
            df[c] = np.nan
    for m in METRICS:
        if m not in df.columns:
            df[m] = np.nan
        else:
            df[m] = pd.to_numeric(df[m], errors="coerce")

    # 2) Relleno específico (LBP si no existe en ciertas temporadas)
    if "player_season_lbp_completed_90" in df.columns:
        df["player_season_lbp_completed_90"] = df["player_season_lbp_completed_90"].fillna(0)

    # 3) Z-score por grupo (p. ej., por temporada)
    Z_parts, scaler_by_group = [], {}
    gcols = list(group_standardization) if group_standardization else []
    if gcols:
        # si hay agrupación 
        for gkey, gdf in df.groupby(gcols, dropna=False):
            X = gdf[METRICS].copy() # solo métricas del grupo
            med = X.median(numeric_only=True) # mediana por columnas
            X = X.fillna(med) # imputa faltantes dentro del grupo
            scaler = StandardScaler() # z-score por grupo
            Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=gdf.index) # estandarizacion de variables
            gZ = pd.concat([gdf[ID_COLS], Xz.add_prefix("z_")], axis=1)
            Z_parts.append(gZ)
            scaler_by_group[gkey if isinstance(gkey, tuple) else (gkey,)] = scaler
        Zall_new = pd.concat(Z_parts, ignore_index=True)
    else:
        X = df[METRICS].copy()
        med = X.median(numeric_only=True)
        X = X.fillna(med)
        scaler = StandardScaler()
        Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=df.index)
        Zall_new = pd.concat([df[ID_COLS], Xz.add_prefix("z_")], axis=1)
        scaler_by_group = {("GLOBAL",): scaler}

    # 4) PCA con K automático (usa tu select_pca_k existente)
    Xz_all = Zall_new[[f"z_{m}" for m in METRICS]].to_numpy() # construye la matriz solo con las columnas z-estandarizadas en el orden de metrics
    k, info = select_pca_k(Xz_all, method=k_method, variance_threshold=variance_threshold, random_state=random_state) # llama al selector de componentes, devuelve k y info 
    pca = PCA(n_components=k, random_state=random_state).fit(Xz_all) # ajusta PCA con k componentes
    PCs = pca.transform(Xz_all) # proyecta los datos estandarizados a los componentes principales 
    for i in range(k):
        #añade los PCs como columnas al dataframe final
        Zall_new[f"PC{i+1}"] = PCs[:, i]

    return Zall_new, pca, scaler_by_group, info # regresa Zall_new (IDs, z-scores, pCs), pca (modelo PCA entrenado), scaler_by_group (diccionario de escaladores por grupo), info (metadatos del proceso de selección de k)

In [ ]:
def select_pca_k(Xz, method, variance_threshold=0.85,
                 cv_folds=5, random_state=42, n_iter_parallel=200):
    # Función que determina automáticamente cuantos componentes del PCA usar 
    n_samples, n_features = Xz.shape  # Obtiene dimensiones de la matriz: filas=muestras, columnas=métricas
    max_k = min(n_samples, n_features)  # El máximo k posible es el menor entre muestras y métricas

    pca_full = PCA(n_components=max_k, random_state=random_state).fit(Xz)  # Ajusta PCA con el máximo de componentes
    eigvals = pca_full.explained_variance_  # Autovalores: varianza explicada por cada componente
    prop = pca_full.explained_variance_ratio_  # Proporción de varianza explicada por cada componente
    cumprop = np.cumsum(prop)  # Varianza acumulada a medida que agregas componentes

    if method == "variance":  # Si eliges el criterio de varianza acumulada
        k = int(np.searchsorted(cumprop, variance_threshold) + 1)  # Primer k cuyo acumulado supera el umbral

    elif method == "parallel":  # Si usas Parallel Analysis (Horn)
        rng = np.random.default_rng(random_state)  # Generador de aleatoriedad reproducible
        rand_eigs = np.zeros((n_iter_parallel, len(eigvals)))  # Matriz para guardar autovalores de datos aleatorios
        for b in range(n_iter_parallel):  # Repite simulaciones aleatorias n_iter_parallel veces
            Xb = np.empty_like(Xz)  # Crea matriz vacía con misma forma que Xz
            for j in range(n_features):  # Recorre columnas para romper correlaciones
                Xb[:, j] = rng.permutation(Xz[:, j])  # Baraja la columna j (conserva distribución marginal)
            pca_b = PCA(n_components=max_k, random_state=random_state).fit(Xb)  # PCA sobre datos “sin estructura”
            rand_eigs[b, :len(pca_b.explained_variance_)] = pca_b.explained_variance_  # Guarda autovalores aleatorios
        mean_rand = rand_eigs.mean(axis=0)  # Promedio de autovalores aleatorios por componente
        k = int(np.sum(eigvals > mean_rand)) or 1  # Cuenta cuántos componentes reales superan al azar; al menos 1

    elif method == "broken_stick":  # Si usas el modelo teórico Broken-Stick
        H = np.array([np.sum(1.0/np.arange(j, n_features+1)) for j in range(1, n_features+1)])  # Sumas armónicas
        bs = H / H[0]  # Proporciones esperadas del modelo (normaliza por la primera suma)
        k = int(np.sum(prop >= bs[:len(prop)])) or 1  # Conserva componentes cuya proporción supera la línea teórica

    elif method == "cv":  # Si usas validación cruzada con error de reconstrucción
        kf = KFold(n_splits=cv_folds, shuffle=True, random_state=random_state)  # Define particiones KFold reproducibles
        mse_by_k = []  # Lista para guardar (k, mse_promedio)
        for k_try in range(1, max_k+1):  # Prueba k desde 1 hasta max_k
            mses = []  # Acumula errores por fold para este k
            pca_k = PCA(n_components=k_try, random_state=random_state)  # PCA con k_try componentes
            for tr, te in kf.split(Xz):  # Para cada split de entrenamiento/prueba
                pca_k.fit(Xz[tr])  # Ajusta PCA en el conjunto de entrenamiento
                X_te_proj = pca_k.inverse_transform(pca_k.transform(Xz[te]))  # Reconstruye el conjunto de prueba
                mses.append(np.mean((Xz[te] - X_te_proj)**2))  # Calcula MSE entre original y reconstruido
            mse_by_k.append((k_try, float(np.mean(mses))))  # Guarda el MSE promedio para este k
        k = min(mse_by_k, key=lambda t: t[1])[0]  # Elige el k con menor MSE promedio

    else:  # Si el método no es válido
        raise ValueError("method debe ser: 'variance' | 'parallel' | 'broken_stick' | 'cv'")  # Mensaje de error claro

    return int(k), {  # Devuelve k y un diccionario con detalles del ajuste
        "eigvals": eigvals,  # Autovalores reales (útil para gráficos/diagnóstico)
        "prop": prop,  # Proporción de varianza por componente
        "cumprop": cumprop,  # Varianza acumulada por número de componentes
        "selected_k": int(k),  # Número de componentes seleccionado
        "method": method  # Método utilizado para seleccionar k
    }

In [ ]:
def build_fingerprint_df_autoK(statsJugadorPorTemporada, competition_id=73, min_minutes=900,
                               k_method="parallel", variance_threshold=0.85,
                               group_standardization=("season_id",), random_state=42):
    # Arma el Zall y aplica PCA con k óptimo
    df = concat_player_season_stats(statsJugadorPorTemporada, competition_id=competition_id)  # Une temporadas en un DF
    df = filter_by_minutes(df, min_minutes=min_minutes)  # Filtra jugadores con pocos minutos
    df["player_season_lbp_completed_90"] = df["player_season_lbp_completed_90"].fillna(0)  # Si falta LBP, pon 0

    for c in ID_COLS:  # Recorre las columnas de identificación mínimas requeridas
        if c not in df.columns:  # Si alguna no existe en el DF original
            df[c] = np.nan  # Créala como NaN para evitar errores posteriores

    Z_parts, scaler_by_group = [], {}  # Inicializa lista de partes normalizadas y dict de escaladores por grupo
    gcols = list(group_standardization) if group_standardization else []  # Determina columnas de agrupación
    if gcols:  # Si hay agrupación (p.ej., por temporada)
        for gkey, gdf in df.groupby(gcols, dropna=False):  # Itera por grupo (temporada, o comp+temporada)
            X = gdf[METRICS].copy()  # Toma solo métricas
            med = X.median(numeric_only=True)  # Calcula medianas por métrica (para imputar)
            X = X.fillna(med)  # Imputa NaN con mediana (robusto a outliers)
            scaler = StandardScaler()  # Crea estandarizador para z-score
            Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=gdf.index)  # Aplica z-score al grupo
            gZ = pd.concat([gdf[ID_COLS], Xz.add_prefix("z_")], axis=1)  # Junta IDs con z_métricas
            Z_parts.append(gZ)  # Agrega el bloque normalizado a la lista
            scaler_by_group[gkey if isinstance(gkey, tuple) else (gkey,)] = scaler  # Guarda el scaler del grupo
        Zall = pd.concat(Z_parts, axis=0, ignore_index=True)  # Concatena todos los grupos normalizados
    else:  # Si no hay agrupación (normalización global)
        X = df[METRICS].copy()  # Toma métricas completas
        med = X.median(numeric_only=True)  # Medianas para imputar
        X = X.fillna(med)  # Imputa NaN
        scaler = StandardScaler()  # Estandarizador global
        Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=df.index)  # z-score global
        Zall = pd.concat([df[ID_COLS], Xz.add_prefix("z_")], axis=1)  # Concatena IDs + z_métricas
        scaler_by_group[("GLOBAL",)] = scaler  # Guarda el scaler global

    Xz_all = Zall[[f"z_{m}" for m in METRICS]].to_numpy()  # Extrae matriz numpy solo con z_métricas para PCA
    k, info = select_pca_k(  # Llama al selector automático para decidir el número óptimo de componentes
        Xz_all,
        method=k_method,  # Método elegido (por defecto 'parallel')
        variance_threshold=variance_threshold,  # Umbral si usaras 'variance'
        random_state=random_state  # Semilla para reproducibilidad
    )

    pca = PCA(n_components=k, random_state=random_state).fit(Xz_all)  # Ajusta PCA con k óptimo
    PCs = pca.transform(Xz_all)  # Proyecta los datos a los k componentes principales
    for i in range(k):  # Para cada componente calculado
        Zall[f"PC{i+1}"] = PCs[:, i]  # Crea columna PCi en Zall con el valor del componente para cada fila

    return Zall, pca, scaler_by_group, info  # Devuelve DF con PCs, el modelo PCA, escaladores y detalles de k

In [ ]:
def similares(Zall, player_id, k=10):
    # Función que encuentra los k jugadores más similares a partir de un player_id
    zcols = [c for c in Zall.columns if c.startswith("z_")] # lista de columnas de caracteristicas
    row = Zall.loc[Zall["player_id"]==player_id, zcols] # extrae, para el player_id, solo la columna z_
    if row.empty: raise ValueError(f"player_id {player_id} no encontrado.")
    # si no se encontro el jugador, lanza un error claro
    v = row.to_numpy()[0] # convierte esa fila a vector numpy
    M = Zall[zcols].to_numpy() # convierte todas las zcols de Zall en una matriz
    cos = (M @ v) / (norm(M,axis=1)*norm(v)) # calcula cosine similarity entre v y cada fila de M
    out = Zall[["player_id","player_name","team_id","team_name","season_id","season_name"]].copy() # crea un dataframe de salida con metadatos básicos de cada jugador
    out["cosine"] = cos # añade la columna cosine con la similitud calculada
    return out[out["player_id"]!=player_id].nlargest(k, "cosine")

In [ ]:
def get_team_members(statsJugadorPorTemporada, season_id, team_id):
    # Función que devuelve el roster de jugadores de un equipo para una temporada
    keys = [k for k in statsJugadorPorTemporada.keys() if k[0]==season_id] # recorre todas las llaves del diccionario y se queda con las que tengan como primer elemento el id de la temporada
    if not keys: raise ValueError(f"No hay stats para season_id={season_id}.")
    # si no hay ninguna llave coincidente, lanza un error claro: no hay datos para esa temporada
    key = keys[0] # toma la primera llave coincidente 
    df = statsJugadorPorTemporada[key].copy() # recupera el dataframe de esa temporada y hace una copia para no modificar el original
    rename_map = {} # diccionario del mapeo para renombrar columnas a un esquema estandar
    # para fuentes con nombres distintos arma el mapeo, cada renombre solo se agrega si la columna fuente existe y la columna destino aún no existe
    if "name" in df.columns and "player_name" not in df.columns: rename_map["name"] = "player_name" 
    if "team" in df.columns and "team_name" not in df.columns: rename_map["team"] = "team_name"
    if "teamId" in df.columns and "team_id" not in df.columns: rename_map["teamId"] = "team_id"
    if "playerId" in df.columns and "player_id" not in df.columns: rename_map["playerId"] = "player_id"
    if rename_map: df = df.rename(columns=rename_map) # si hay algo que renombrar --> aplica el renombrado al dataframe
    if "team_id" not in df.columns: raise ValueError("No se encontró 'team_id' en player_season_stats.") # team_id no se puede filtrar por equipo, por lo que se aborta 
    roster = df[df["team_id"]==team_id][["player_id","player_name","team_id","team_name"]].drop_duplicates() # filtra filas cuyo team_id coincide con el solicitado
    return roster.sort_values("player_name").reset_index(drop=True) # ordena el roster alfabéticamente por player_name

In [ ]:
# 0) Ruta a tu CSV de posiciones (usa r"" en Windows para backslashes)
pos_path = r"tpi_player_season_ligamx.csv" # cadena con la ruta que contiene las posiciones por jugador/temporada

# 1) Une temporadas como siempre
all_stats = concat_player_season_stats(statsJugadorPorTemporada, competition_id=73) # unir en un solo DF tdas las stats por temporada que están en el diccionario

# 2) Adjunta posiciones desde tu CSV y ELIMINA PORTEROS
all_stats = attach_positions(all_stats, pos_path) 
all_stats_no_gk = drop_goalkeepers(all_stats)

# Verificacion cuántos eliminaste
n_before = len(all_stats)
n_after  = len(all_stats_no_gk)
print(f"Filas totales con posición: {n_before} | Sin porteros: {n_after} | Eliminados: {n_before - n_after}")

# 3) Filtro de minutos (como ya hacías)
all_stats_no_gk = filter_by_minutes(all_stats_no_gk, min_minutes=900)

# 4) Construye Zall DESDE este DF ya filtrado (sin porteros)
Zall, pca, scalers, kinfo = build_fingerprint_df_autoK_FROM_DF(
    all_stats_no_gk,
    k_method="parallel",
    variance_threshold=0.85,
    group_standardization=("season_id",),
    random_state=42
)

print("Filas (jugadores-temporadas) Sin porteros y con minutos:", len(Zall))
print("Varianza explicada PCA:", pca.explained_variance_ratio_)
Zall.head()


Filas totales con posición: 2820 | Sin porteros: 2706 | Eliminados: 114
Filas (jugadores-temporadas) Sin porteros y con minutos: 1508
Varianza explicada PCA: [0.39125228 0.26336449 0.08289817]


,player_id,player_name,team_id,team_name,competition_id,season_id,season_name,z_player_season_passing_ratio,z_player_season_pass_length,z_player_season_lbp_completed_90,...,z_player_season_tackles_90,z_player_season_interceptions_90,z_player_season_aerial_wins_90,z_player_season_padj_tackles_and_interceptions_90,z_player_season_dribbles_90,z_player_season_carries_90,z_player_season_deep_progressions_90,PC1,PC2,PC3
0,13034,Sebastián Saucedo Mondragón,1301,Pumas UNAM,73,108,2021/2022,-0.469687,-0.193116,-0.772879,...,-0.318935,-0.830877,-1.409879,-0.908478,1.282707,0.635200,-0.094918,2.471902,1.712512,0.198077
1,30458,Alejandro Zendejas Saavedra,1221,Necaxa,73,108,2021/2022,0.032157,-0.044561,0.214481,...,0.739356,0.708783,-0.596587,0.707159,0.675471,0.751720,0.952881,0.493582,2.376083,-1.325327
2,30458,Alejandro Zendejas Saavedra,1221,Necaxa,73,108,2021/2022,0.032157,-0.044561,0.214481,...,0.739356,0.708783,-0.596587,0.707159,0.675471,0.751720,0.952881,0.493582,2.376083,-1.325327
3,28529,Fernando David Arce Juárez,1291,Juárez,73,108,2021/2022,1.080911,0.086801,-0.060917,...,1.008524,1.947622,-0.429170,1.237603,-0.514283,0.061636,-0.793268,-2.271331,-0.271488,-2.384620
4,30458,Alejandro Zendejas Saavedra,1229,América,73,108,2021/2022,-0.025801,-0.925593,-0.314916,...,0.311967,-0.110381,-0.889628,0.088317,0.690732,0.579485,0.192812,1.892418,1.851183,-0.576871


In [ ]:
# VERIFICACIÓN DE DATOS: 
player_id_objetivo = 13034  # cambia por un id real que veas en Zall["player_id"]
sim_df = similares(Zall, player_id=player_id_objetivo, k=10)
sim_df
season_id = 317  # 2024/2025
team_id = 1299   # pon el id real del club

roster = get_team_members(statsJugadorPorTemporada, season_id=season_id, team_id=team_id)
roster

,player_id,player_name,team_id,team_name,season_id,season_name,cosine
374,5637,Edgar Yoel Bárcenas Herrera,2773,Mazatlán,235,2022/2023,0.898391
1351,406131,Owen de Jesús González Ojeda,1298,Pachuca,317,2024/2025,0.892668
246,28418,Ían Jairo Misael Torres Ramírez,1296,Atlas,108,2021/2022,0.889027
247,28418,Ían Jairo Misael Torres Ramírez,1296,Atlas,108,2021/2022,0.889027
306,28528,Jean David Meneses Villarroel,1302,León,108,2021/2022,0.883914
307,28528,Jean David Meneses Villarroel,1302,León,108,2021/2022,0.883914
81,27899,Brian Avelino Lozano Aparicio,1299,Santos Laguna,108,2021/2022,0.881104
770,27228,Kevin Andrés Velasco Bonilla,1226,Puebla,281,2023/2024,0.874103
30,27356,Nicolás Benedetti Roa,2773,Mazatlán,108,2021/2022,0.867994
1147,27356,Nicolás Benedetti Roa,2773,Mazatlán,317,2024/2025,0.847853


In [ ]:
# ===============================================
# PCA:
# ===============================================
# Matriz de loadings: cada fila = componente, cada columna = métrica
# Forma: (n_components, n_features)
loadings = pd.DataFrame(
    # creacion de un DataFrame
    pca.components_, # matriz de tamaño
    index=[f"PC{i+1}" for i in range(pca.n_components_)], # cada fila es el vector propio de una componente principal en el espacio de las variables estandarizadas
    columns=METRICS # utiliza la columna metric para etiquetar cada loading por metrica
) # cada número es el loading de esa métrica en esa PC

# Correlación PC–métricas: corr(PCk, z_metrica)
# Usamos que PCs = Z * loadings^T (ortogonales), así que la correlación ~ loading * sqrt(var_exp)
# Para no depender de aproximaciones, la computamos empíricamente:
zcols = [f"z_{m}" for m in METRICS] # nombres de columnas en Zall
PCcols = [f"PC{i+1}" for i in range(pca.n_components_)] # nombre de columnas ya calculadas
corrs = pd.DataFrame(index=PCcols, columns=METRICS, dtype=float) # dataframe vacío con PCs como filas y métricas como columnas

for pc in PCcols:
    # doble bucle para llenar la matriz de correlaciones 
    for m, zm in zip(METRICS, zcols): 
        corrs.loc[pc, m] = np.corrcoef(Zall[pc], Zall[zm])[0,1] # ceoficiente de correlación de Pearson entre Zall[pc] y Zall[zm]

def top_features_for_pc(pc_name, k=6):
    # Ordena y muestra top ± por PC    
    s = corrs.loc[pc_name].dropna() # fila de corrs para esa PC 
    top_pos = s.sort_values(ascending=False).head(k) # las k métricas con mayor correlación positiva
    top_neg = s.sort_values(ascending=True).head(k) # las k con mayor correlación negativa 
    return top_pos, top_neg # devuelve dos series con los mejores drivers en ambos signos 

for pc in PCcols:
    # para cada PC
    pos, neg = top_features_for_pc(pc, k=6) # calcula sus k métricas más correlacionadas en positivo y negativo
    print(f"\n=== {pc} ===") # nombre de la PC
    print("↑ Positivos (correlan +):") # valores de correlacion positivo formateado
    print(pos.to_string())
    print("↓ Negativos (correlan -):") # valores de correlacion negativo formateado
    print(neg.to_string())



=== PC1 ===
↑ Positivos (correlan +):
player_season_touches_inside_box_90    0.877612
player_season_np_shots_90              0.875949
player_season_np_xg_90                 0.848595
player_season_xa_90                    0.662341
player_season_key_passes_90            0.611243
player_season_dribbles_90              0.567114
↓ Negativos (correlan -):
player_season_interceptions_90                    -0.834294
player_season_padj_tackles_and_interceptions_90   -0.731360
player_season_pass_length                         -0.703202
player_season_passing_ratio                       -0.696164
player_season_lbp_completed_90                    -0.633464
player_season_tackles_90                          -0.527996

=== PC2 ===
↑ Positivos (correlan +):
player_season_deep_progressions_90     0.872202
player_season_carries_90               0.822621
player_season_op_passes_into_box_90    0.732748
player_season_key_passes_90            0.677449
player_season_xa_90                    0.572902
player_s

In [ ]:
# diccionario de las métricas indicativas para un aporte ofensivo
OFFENSIVE = {
    "player_season_np_shots_90",            # número de tiros de juego abierto
    "player_season_np_xg_90",               # expected goals acumulada de esos tiros de juego abierto por cada 90 min
    "player_season_touches_inside_box_90",  # número de toques del jugador dentro del área del adversario por cada 90 min
    "player_season_key_passes_90",          # pases que crean una clara oportunidad de tiro para un compañero por cada 90 min
    "player_season_xa_90",                  # expected assists por cada 90 min
    "player_season_op_passes_into_box_90",  # pases ofensivos del jugador que entran al área del adversario por cada 90 min
    "player_season_dribbles_90",            # números de regates exitosos por cada 90 min
    "player_season_carries_90",             # numero de desplazamientos con balon en los pies por cada 90 min
    "player_season_deep_progressions_90",   # numero de progresiones profundas por cada 90 min
    "player_season_lbp_completed_90",       # line breaking passes completados por cada 90 min
}

def orient_pc_signs(Zall, pca, corrs, offensive_set=OFFENSIVE):
    # Funcion que orienta el signo de cada PC para que las PCs tengan correlación positiva con las métricas ofensivas
    PCcols = [f"PC{i+1}" for i in range(pca.n_components_)] # construye la lista de nombres de columnas de las PCs
    flipped = {} # diccionario que registra si cad PC fue volteada
    for i, pc in enumerate(PCcols):
        # itera sobre cada PC con su índice i y su nombre pc
        s = corrs.loc[pc, list(offensive_set & set(METRICS))].dropna() # toma la fila de corrs correspondientes a esa PC, solo las que esten en offensive_set y METRICS
        score = s.sum() # puntaje de orientacion
        if score < 0:
            # si el puntaje es negativo, se voltea el signo de esa OC
            Zall[pc] = -Zall[pc]
            corrs.loc[pc, :] = -corrs.loc[pc, :] 
            pca.components_[i, :] = -pca.components_[i, :]
            flipped[pc] = True # registro de esa PC
        else:
            flipped[pc] = False
    return flipped

flipped_info = orient_pc_signs(Zall, pca, corrs, OFFENSIVE) # llama a la funcion usando el conjunto ofensivo por defecto
print("PCs volteados por convención ofensiva:", flipped_info)


PCs volteados por convención ofensiva: {'PC1': False, 'PC2': False, 'PC3': False}


In [ ]:
LABEL_RULES = [
    # lista de reglas: cada regla es un diccionario con nombre y metricas positivas y negativas
    {
        "name": "Aporte Ofensivo vs Defensivo",
        "positives": {"player_season_np_shots_90","player_season_np_xg_90",
                      "player_season_touches_inside_box_90"},
        "negatives": {"player_season_tackles_90","player_season_interceptions_90"}
    },
    {
        "name": "Creador Asociativo vs Juego Directo",
        "positives": {"player_season_key_passes_90","player_season_xa_90",
                      "player_season_lbp_completed_90","player_season_op_passes_into_box_90"},
        "negatives": {"player_season_pass_length"}
    },
    {
        "name": "Intensidad de Presión/Recuperación",
        "positives": {"player_season_pressures_90","player_season_tackles_90",
                      "player_season_interceptions_90","player_season_padj_tackles_and_interceptions_90"},
        "negatives": set()
    },
    {
        "name": "Progresión en Conducción (1v1)",
        "positives": {"player_season_dribbles_90","player_season_carries_90","player_season_deep_progressions_90"},
        "negatives": set()
    },
    {
        "name": "Dominio Aéreo",
        "positives": {"player_season_aerial_wins_90"},
        "negatives": set()
    }
]

def score_label_rule(pc_name, corrs, rule):
    # Función que puntúa una PC dada contra una regla usando la matriz de correlaciones
    pos = corrs.loc[pc_name, list(rule["positives"] & set(METRICS))].sum() if rule["positives"] else 0.0 # calcula el aporte positivo
    neg = -corrs.loc[pc_name, list(rule["negatives"] & set(METRICS))].sum() if rule["negatives"] else 0.0 # calcula el aporte negativo
    return pos + neg #ndevuelve el score total de esa PC para esa regla

def auto_label_components(corrs, rules=LABEL_RULES):
    # recorre todas las OCs y les asigna la mejor etiqueta 
    labels = {} # diccionario de salida
    for pc in corrs.index:
        # itera sobre cada PC
        scores = [(score_label_rule(pc, corrs, r), r["name"]) for r in rules] # construye una lista de tuplas culculando la puntuacion de esa PC con cada regla
        scores.sort(reverse=True) # ordena de mayor a menor por score
        best = scores[0] # toma la mejor regla
        labels[pc] = {"label": best[1], "score": float(best[0])} # asigna el nombre de la regla con mayor score
    return labels # devuelve el mapeo completo

auto_labels = auto_label_components(corrs, LABEL_RULES) # ejecución del etiquetado automático
auto_labels


{'PC1': {'label': 'Aporte Ofensivo vs Defensivo', 'score': 3.9644460085537307},
 'PC2': {'label': 'Creador Asociativo vs Juego Directo',
  'score': 2.5695803604657743},
 'PC3': {'label': 'Aporte Ofensivo vs Defensivo', 'score': 0.7177892307578715}}

In [ ]:
def pc_summary(pc, k=5):
    # funcion que genera un resumen textual para un componente principal 
    s = corrs.loc[pc].dropna() # toma la fila de corrs correspondiente a la PC
    top_pos = s.sort_values(ascending=False).head(k) # ordena las correlaciones de mayor a menor y se queda con las k mas positivas
    top_neg = s.sort_values(ascending=True).head(k) # ordena las correlaciones de menor a mayor y toma las k mas megativas
    lab = auto_labels[pc]["label"] # recupera la etiqueta asignada a esa PC
    text = f"• {pc} — {lab}\n" \
           f"  Señales ↑: {', '.join([f'{m} ({top_pos[m]:+.2f})' for m in top_pos.index])}\n" \
           f"  Señales ↓: {', '.join([f'{m} ({top_neg[m]:+.2f})' for m in top_neg.index])}" # string multilineal con el resumen
    return text #devuelve el texto 

for pc in PCcols:
    # itera por todas las PCs en PCcols e imprime el resumen de cada una
    print(pc_summary(pc, k=4))


• PC1 — Aporte Ofensivo vs Defensivo
  Señales ↑: player_season_touches_inside_box_90 (+0.88), player_season_np_shots_90 (+0.88), player_season_np_xg_90 (+0.85), player_season_xa_90 (+0.66)
  Señales ↓: player_season_interceptions_90 (-0.83), player_season_padj_tackles_and_interceptions_90 (-0.73), player_season_pass_length (-0.70), player_season_passing_ratio (-0.70)
• PC2 — Creador Asociativo vs Juego Directo
  Señales ↑: player_season_deep_progressions_90 (+0.87), player_season_carries_90 (+0.82), player_season_op_passes_into_box_90 (+0.73), player_season_key_passes_90 (+0.68)
  Señales ↓: player_season_aerial_wins_90 (-0.72), player_season_np_xg_90 (-0.23), player_season_touches_inside_box_90 (-0.22), player_season_pass_length (-0.03)
• PC3 — Aporte Ofensivo vs Defensivo
  Señales ↑: player_season_pass_length (+0.40), player_season_lbp_completed_90 (+0.25), player_season_carries_90 (+0.24), player_season_passing_ratio (+0.17)
  Señales ↓: player_season_pressures_90 (-0.73), player_

In [ ]:
def varimax(Phi, gamma = 1.0, q = 20, tol = 1e-6):
    # Función que realiza la rotación ortogonal clásica de Kaiser
    p,k = Phi.shape # extrae p y k
    R = np.eye(k) # inicialliza la matriz de rotación ortogonal
    d= 0 # escalar objetivo
    for i in range(q):
        # bucle hasta q iteraciones
        d_old = d # guarda el valor anterior del objetivo para 
        Lambda = Phi @ R
        u,s,vh = np.linalg.svd(Phi.T @ (Lambda**3 - (gamma/p) * Lambda @ np.diag(np.diag(Lambda.T @ Lambda))))
        R = u @ vh
        d = np.sum(s)
        if d_old!=0 and d/d_old < 1 + tol: break
    return Phi @ R, R

# Aplica varimax a los loadings originales
L = pca.components_.T  # (features, components)
Lv, R = varimax(L)     # rotated loadings
rot_loadings = pd.DataFrame(Lv.T, index=PCcols, columns=METRICS) # vuelve a transponer a kxp para tener filas PCs y columnas metricas

In [ ]:
PC_COLS = [f"PC{i+1}" for i in range(pca.n_components_)] # lista con los nombres de las columnas de los componentes principales segun cuantos componentes tenga el pca

def _get_Xpcs(Zall, scale_pc=True, pc_cols=None):
    # funcion auxiliar que regresa una matriz con los scores de PCs extraidos
    if pc_cols is None:
        # si no especificaron columnas, detecta todas las columnas de Zall que empiecen con PC
        pc_cols = [c for c in Zall.columns if c.startswith("PC")] 
    X = Zall[pc_cols].to_numpy().copy() # selecciona de Zall solo las columnas de PC y las convierte a un arreglo
    if scale_pc:
        # si scale_pc es true, estandariza cada componente columna a media 0 y desviación estandar 1
        X = (X - X.mean(axis=0)) / (X.std(axis=0, ddof=0) + 1e-12)
    return X # devuelve la matriz X

In [ ]:
def auto_select_k(Zall, k_min=2, k_max=15, *, method="silhouette",
                  scale_pc=True, pc_cols=None, random_state=42, n_init=50):
    """
    Elige k automáticamente con uno de:
      - 'silhouette' (↑ mejor)
      - 'ch'         (Calinski–Harabasz ↑)
      - 'db'         (Davies–Bouldin ↓)
      - 'ensemble'   (votación entre silhouette, CH y DB; desempata con silhouette, luego CH, luego menor k)
    Devuelve: k_opt, justificación, diag(DataFrame con métricas por k)
    """
    X = _get_Xpcs(Zall, scale_pc=scale_pc, pc_cols=pc_cols) # extrae la matriz con las PCs
    rows = [] # lista de diccionarios con métricas para cada k
    for k in range(k_min, k_max+1):
        # itera todas las opciones de k en el rango
        km = KMeans(n_clusters=k, random_state=random_state, n_init=n_init).fit(X) # ajusta KMeans con k clusters sobre X
        labels = km.labels_ # etiquetas de cluster asignadas a cada fila de X
        rows.append({
            # concatena para ese k 
            "k": k,
            "inertia": float(km.inertia_), # suma de distancias intra-cluster
            "silhouette": float(silhouette_score(X, labels)) if k > 1 else float("nan"), # silueta
            "ch": float(calinski_harabasz_score(X, labels)) if k > 1 else float("nan"), # Calinski-Harabasz
            "db": float(davies_bouldin_score(X, labels)) if k > 1 else float("nan"), # Davies-bouldin
        })
    diag = pd.DataFrame(rows) # convierte la lista de resultados en un dataframe diagnóstico por k

    method = method.lower() # normaliza el método a minusculas
    if method == "silhouette": 
        # seleccion por máximo silhouette
        idx = diag["silhouette"].idxmax()
        k_opt = int(diag.loc[idx, "k"]) # guarda k 
        just = f"Máximo Silhouette = {diag.loc[idx, 'silhouette']:.3f}" # justificacion
    elif method == "ch":
        # máximo Calinski-Harabasz
        idx = diag["ch"].idxmax()
        k_opt = int(diag.loc[idx, "k"])
        just = f"Máximo Calinski–Harabasz = {diag.loc[idx, 'ch']:.1f}"
    elif method == "db":
        # mínimo David-Bouldin
        idx = diag["db"].idxmin()
        k_opt = int(diag.loc[idx, "k"])
        just = f"Mínimo Davies–Bouldin = {diag.loc[idx, 'db']:.3f}"
    elif method == "ensemble":
        # ganador por cada métrica individual
        k_sil = int(diag.loc[diag["silhouette"].idxmax(), "k"])
        k_ch  = int(diag.loc[diag["ch"].idxmax(), "k"])
        k_db  = int(diag.loc[diag["db"].idxmin(), "k"])
        votos = Counter([k_sil, k_ch, k_db]) # votacion: cuenta cuantas veces gano cada k
        max_votos = max(votos.values())
        candidatos = sorted([k for k, v in votos.items() if v == max_votos])  # lista ordenada
        if len(candidatos) == 1:
            # si solo hay un ganador ese es el k optimo
            k_opt = candidatos[0]
        else:
            # desempates: mejor silhouette, luego CH, luego menor k
            best_sil_k = int(diag.loc[diag["silhouette"].idxmax(), "k"])
            if best_sil_k in candidatos:
                k_opt = best_sil_k
            else:
                best_ch_k = int(diag.loc[diag["ch"].idxmax(), "k"])
                if best_ch_k in candidatos:
                    k_opt = best_ch_k
                else:
                    k_opt = min(candidatos)
        just = f"Ensemble (votos: Sil={k_sil}, CH={k_ch}, DB={k_db}) → k={k_opt}" # construye una explicacion del resultado del ensamble
    else:
        raise ValueError("method debe ser: 'silhouette' | 'ch' | 'db' | 'ensemble'") # valida argumento 

    return k_opt, just, diag # devuelve el k elegido, texto con la justiificacion y tabla con métricas por cada k probada

In [ ]:
def fit_kmeans_and_assign(Zall, k, *, scale_pc=True, colname=None, pc_cols=None, random_state=42, n_init=50):
    # Función que entrena KMeans con k clusters sobre las PCs de Zall y asigna las etiquetas al propio Zall
    if colname is None:
        # si no se especifica el nombre para la columna de etiquetas, crea uno 
        colname = f"cluster_k{k}"
    X = _get_Xpcs(Zall, scale_pc=scale_pc, pc_cols=pc_cols) # extrae de Zall la matriz Numpy x con las PCs seleccionadas
    km = KMeans(n_clusters=k, random_state=random_state, n_init=n_init) # instania el modelo KMeans con k cantidad de clusters, random_state para reproducibilidad, n cantidad de reinicios
    labels = km.fit_predict(X) # entrena KMeans sobre x y predice las etiquetas de cluster para cada fila
    Zall[colname] = labels # añade la columna de etiquetas de cluster a Zall con el nombre colname
    return Zall, km # regresa el dataframe enriquecido con la columna de cluster y el objeto KMeans entrenado

In [ ]:
def profile_clusters(Zall, cluster_col):
    """
    Regresa:
      - prof_z: medias de todas las z_ métricas por clúster (para interpretar).
      - size: tamaño de cada clúster.
    """
    zcols = [f"z_{m}" for m in METRICS if f"z_{m}" in Zall.columns]
    grp = Zall.groupby(cluster_col, dropna=False)
    prof_z = grp[zcols].mean().sort_index()
    size = grp.size().rename("n")
    return prof_z, size


In [ ]:
ROLE_RULES = [
    # Recuperación/Presión
    {
        "name_pos": "Mediocampista Recuperador",
        "pos": {"player_season_pressures_90","player_season_tackles_90",
                "player_season_interceptions_90","player_season_padj_tackles_and_interceptions_90"},
        "neg": set()
    },
    # Extremo desequilibrante
    {
        "name_pos": "Extremo Desequilibrante",
        "pos": {"player_season_dribbles_90","player_season_carries_90",
                "player_season_deep_progressions_90"},
        "neg": set()
    },
    # Creador Asociativo
    {
        "name_pos": "Creador Asociativo",
        "pos": {"player_season_key_passes_90","player_season_xa_90",
                "player_season_lbp_completed_90","player_season_op_passes_into_box_90"},
        "neg": {"player_season_pass_length"}
    },
    # Finalizador/9 de área
    {
        "name_pos": "Delantero de Área",
        "pos": {"player_season_np_shots_90","player_season_np_xg_90",
                "player_season_touches_inside_box_90"},
        "neg": set()
    },
    # Central dominante aéreo
    {
        "name_pos": "Defensa Central Aéreo",
        "pos": {"player_season_aerial_wins_90"},
        "neg": set()
    },
    # Lateral/progresión mixta
    {
        "name_pos": "Lateral Progresivo",
        "pos": {"player_season_deep_progressions_90","player_season_op_passes_into_box_90"},
        "neg": set()
    },
]

def score_rule(cluster_mean, rule):
    s_pos = cluster_mean[[f"z_{m}" for m in rule["pos"] if f"z_{m}" in cluster_mean.index]].sum()
    s_neg = -cluster_mean[[f"z_{m}" for m in rule["neg"] if f"z_{m}" in cluster_mean.index]].sum() if rule["neg"] else 0.0
    return float(s_pos + s_neg)

def auto_name_clusters(prof_z):
    """
    prof_z: DataFrame (cluster x z_metrics_mean)
    Devuelve dict {cluster_label: {'role': str, 'top_pos': list, 'top_neg': list}}
    """
    out = {}
    for cl in prof_z.index:
        row = prof_z.loc[cl]
        # ranking de reglas
        scored = [(score_rule(row, r), r["name_pos"]) for r in ROLE_RULES]
        scored.sort(reverse=True)
        role = scored[0][1]
        # features top para explicación
        s = row.sort_values(ascending=False)
        top_pos = s.head(5).index.tolist()
        top_neg = row.sort_values(ascending=True).head(5).index.tolist()
        out[cl] = {
            "role": role,
            "top_pos": top_pos,
            "top_neg": top_neg
        }
    return out


In [ ]:
def cluster_and_label_roles_auto(Zall, *, k_min=2, k_max=15, method="silhouette",
                                 scale_pc=True, pc_cols=None, random_state=42, n_init=50):
    
    k_opt, just, diag = auto_select_k(
        Zall, k_min=k_min, k_max=k_max, method=method,
        scale_pc=scale_pc, pc_cols=pc_cols, random_state=random_state, n_init=n_init
    )
    cluster_col = f"cluster_k{k_opt}"
    Zall, km = fit_kmeans_and_assign(
        Zall, k_opt, scale_pc=scale_pc, colname=cluster_col,
        pc_cols=pc_cols, random_state=random_state, n_init=n_init
    )
    prof_z, size = profile_clusters(Zall, cluster_col)
    names = auto_name_clusters(prof_z)
    Zall["Rol"] = Zall[cluster_col].map({cl: info["role"] for cl, info in names.items()})
    return Zall, km, prof_z, size, names, k_opt, just, diag


In [ ]:
# Selección automática de k y clustering + roles
Zall, km, prof_z, size, names, k_opt, just, diag = cluster_and_label_roles_auto(
    Zall,
    k_min=2, k_max=15,
    method="ensemble",      # o "silhouette", "ch", "db"
    scale_pc=True,
    pc_cols=PC_COLS,
    random_state=42, n_init=50
)
print("size: ",size)
print("hola")

# >>> AHORA sí: corre tu línea para ver los roles disponibles en la temporada 317
roles_disponibles = sorted(Zall.loc[Zall["season_id"]==317, "Rol"].dropna().unique())
print(roles_disponibles)


print(Zall["Rol"])
print("Tamaños por clúster:\n", size)
print("\nNombres asignados:")
for cl, info in names.items():
    print(f"  Cluster {cl}: {info['role']}  | Señales↑ {info['top_pos'][:3]}  | Señales↓ {info['top_neg'][:3]}")

# 3) Guarda resultados
Zall.to_csv(f"Zall_roles_k{k_opt}.csv", index=False)
prof_z.to_csv(f"perfil_z_k{k_opt}.csv")
size.to_frame().to_csv(f"cluster_sizes_k{k_opt}.csv")


size:  cluster_k4
0    488
1    216
2    437
3    367
Name: n, dtype: int64
hola
['Creador Asociativo', 'Defensa Central Aéreo', 'Delantero de Área', 'Mediocampista Recuperador']
0              Creador Asociativo
1       Mediocampista Recuperador
2       Mediocampista Recuperador
3       Mediocampista Recuperador
4              Creador Asociativo
                  ...            
1503           Creador Asociativo
1504           Creador Asociativo
1505           Creador Asociativo
1506           Creador Asociativo
1507           Creador Asociativo
Name: Rol, Length: 1508, dtype: object
Tamaños por clúster:
 cluster_k4
0    488
1    216
2    437
3    367
Name: n, dtype: int64

Nombres asignados:
  Cluster 0: Mediocampista Recuperador  | Señales↑ ['z_player_season_tackles_90', 'z_player_season_padj_tackles_and_interceptions_90', 'z_player_season_pressures_90']  | Señales↓ ['z_player_season_np_xg_90', 'z_player_season_touches_inside_box_90', 'z_player_season_np_shots_90']
  Cluster 1: Dela

In [ ]:
print("Jugadores-temporada en Zall:", len(Zall))
print("¿Columna Rol existe?", "Rol" in Zall.columns)
print("Roles detectados:", sorted(Zall["Rol"].dropna().unique()))
print("Temporada 317 - conteo por Rol:")
print(Zall[Zall["season_id"]==317]["Rol"].value_counts())

Jugadores-temporada en Zall: 1508
¿Columna Rol existe? True
Roles detectados: ['Creador Asociativo', 'Defensa Central Aéreo', 'Delantero de Área', 'Mediocampista Recuperador']
Temporada 317 - conteo por Rol:
Defensa Central Aéreo        128
Mediocampista Recuperador    122
Creador Asociativo            89
Delantero de Área             50
Name: Rol, dtype: int64


In [ ]:
print("Filas totales (jugador-temporada):", len(Zall))
print("Jugadores únicos:", Zall["player_id"].nunique())

Filas totales (jugador-temporada): 1508
Jugadores únicos: 536
